## RNN 创作古诗
在这一章中我们了解到循环神经网络非常擅长处理序列和自然语言处理，文本都是由单词或者汉字按照序列顺序组成的，那么如何能够生成文本呢？下面我们来讲一讲原理，需要你根据这个原理来实现整个网络。

### 原理介绍
前面我们介绍过 RNN 的输入和输出存在多种关系，比如多个输入对一个输出，这个时候输入是一个序列，输出是一个分类结果，就像使用 RNN 做图像分类。

这里我们使用 RNN 来生成文本，网络的输入是一个序列，同时输出也是一个相同长度的序列，结构如下

<img src=https://ws1.sinaimg.cn/large/006tNc79gy1fob5kq3r8jj30mt09dq2r.jpg width=700>

在上面的网络流程中，输入是一个序列 "床 前 明 月 光"，输出也是一个序列 "前 明 月 光 床"。如果你仔细观察可以发现网络的每一步输出都是下一步的输入，这就是其设计思路。

那么对于任意的一段话，比如 "我喜欢小猫"，我们可以将其拆分 "我 喜 欢 小 猫" 这个长度为 5 的序列，网络的每一步输出就是 "喜 欢 小 猫 我"，也就是每个字符的输出就是其**紧跟**的后一个字符。

当然对于一个序列，其最后一个字符后面并没有其他的字符，所以有多种方式选择，比如将序列的第一个字符作为其输出，也就是 "光" 的输出是 "床"，或者将其本身作为输出，也就是 "光" 的输出是 "光"，这里的选择可以有很多，我们使用一种循环的连接，将第一个字符作为最后一个字符的输出。

### 生成文本
这样设计网络的训练流程是为了非常好地生成文本，下面我们说明一下如何进行文本的生成。

首先需要输入网络一段初始的序列进行预热，预热的过程并不需要实际的输出结果，只是为了生成拥有记忆效果的隐藏状态，并将隐藏状态保留下来，接着我们开始正式生成文本，每个字符作为输入都可以得到输出，然后将输出作为下一步的输入，这样就可以不断地生成新的句子，这个过程是可以无限循环下去，或者到达我们的要求输出长度，具体可以看看下面的图示

<img src=https://ws2.sinaimg.cn/large/006tNc79gy1fob5z06w1uj30qh09m0sl.jpg width=800>

讲完了原理之后，下面就该你亲自动手去实现这个网络

首先我们可以探索一下数据集是什么样的

In [ ]:
with open('./dataset/poetry.txt', 'r') as f:
    poetry_corpus = f.read()

我们取得了前100个字符的结果，其中 `\n` 表示换行符

In [ ]:
poetry_corpus[:100]

In [ ]:
# 看看字符数
print('总的字符数: {}'.format(len(poetry_corpus)))

为了可视化比较方便，我们将换行字符 `\n` 替换成空格

In [ ]:
poetry_corpus = poetry_corpus.replace('\n', ' ').replace('\r', ' ').replace('，', ' ').replace('。', ' ')
poetry_corpus[:100]

### 文本数值表示
对于每个文字，电脑并不能像人一样能够有效地识别，所以必须做一个转换，将文字转换成电脑能够识别的数字，相当于每个不同的汉字，都用不同的数字去表示，可以对所有非重复的字符，从 0 开始建立索引

同时可能古诗中会出现一些生僻的字，这些字可能只会出现几次，甚至只会出现一次，引入这些字会增大模型的复杂度，同时也会影响模型的训练，可以将这些词频比较低的字去掉

关于汉字和数字的转换，我们已经为你实现好了一个转换器，感兴趣的同学可以去 `utils.py` 中查看，在之后的练习中，你可以使用这个转换器进行生成文本的转换，下面我们先看看例子

In [ ]:
import numpy as np
from utils import TextConverter

In [ ]:
convert = TextConverter('./dataset/poetry.txt', max_vocab=10000)

上面我们通过数据集建立好了这个转换器 `convert`，下面我们看看如何去调用

In [ ]:
# 得到原始的文本结果
txt_char = poetry_corpus[:11]
print('原始的文本结果: {}'.format(txt_char))
print()

# 通过 convert 将文字转换成字符
num_char = convert.text_to_arr(txt_char)
print('转换成数字之后的结果: {}'.format(num_char))
print()

# 通过 convert 将数字转换成文字
origin_txt_char = convert.arr_to_text(num_char)
print('将数字重新转换成文字: {}'.format(origin_txt_char))

通过上面的例子，你可以看到，能够使用 `convert.text_to_arr` 对一个文本进行数字的转换，通过 `convert.arr_to_text` 将数字转换成文本 

### 构造时序样本数据
对于一整段文本，并不适合全部输入到循环神经网络中，因为我们前面了解到循环神经网络存在着长时依赖的问题，所以需要将整个文本分成很多个序列文本，然后将这些序列文本输入到循环神经网络中进行训练，只要我们定好每个序列的长度，那么序列个数也就被决定了。

In [ ]:
# 每个序列的长度，你可以自行修改
n_step = 20

# 总的序列个数
num_seq = int(len(poetry_corpus) / n_step)

# 去掉最后不足一个序列长度的部分
text = poetry_corpus[:num_seq*n_step]

print('序列的个数: {}'.format(num_seq))

接着需要将序列中所有的文字转换成数字表示，同时重新排列成 **$(num\_seq \times n\_step)$** 的矩阵

In [ ]:
import torch

完成下面的 `#todo` 的部分

In [ ]:
arr = None #todo: 使用 convert 将文本 text 转换成数字表示的数组
arr = None #todo: 将转换之后的数组重新排列成 (num_seq x n_step) 的形状
arr = None #todo: 将数组转换成 pytorch tensor

In [ ]:
# 不要修改下面的代码
# ================== test =================
if arr.shape == (num_seq, n_step):
    print('Successful!')
else:
    print('Failed!')

据此，我们可以构建 PyTorch 中的数据读取来训练网络，这里我们将最后一个字符的输出 label 定为输入的第一个字符，也就是"床前明月光"的输出是"前明月光床"，完成下面 #todo 的部分

**注意：这个地方有困难的同学，可以看看灵活的数据读取这一章的图文教程**

In [ ]:
class TextDataset(object):
    def __init__(self, arr):
        self.arr = arr
        
    def __getitem__(self, item):
        #todo: 取得 arr 中的 item 这一个序列
        
        #todo: 构造上述描述的 label
        
        return x, y
    
    def __len__(self):
        return self.arr.shape[0]

如果你构造好了这个数据集类，我们可以将其实例化

In [ ]:
train_set = TextDataset(arr)

下面我们可以取出其中一个数据集参看一下是否是我们描述的这样，这个数据集需要像上面描述的一样，请自行检查

In [ ]:
x, y = train_set[0]
print('输入的文字序列 x: {}'.format(convert.arr_to_text(x.numpy())))
print('输出的文字序列 y: {}'.format(convert.arr_to_text(y.numpy())))

### 建立模型
下面我们需要构建这个循环神经网路的网络结构，模型可以定义成非常简单的三层
- 第一层是词嵌入, **nn.Embedding**
- 第二层是 RNN 层, **nn.LSTM (nn.GRU)**
- 第三层是线性层，做分类问题，最后输出预测的字符 **nn.Linear**

只需要按照提示填写下面的 #todo 部分

In [ ]:
use_gpu = True

In [ ]:
from torch import nn
from torch.autograd import Variable

class CharRNN(nn.Module):
    def __init__(self, num_classes, embed_dim, hidden_size, 
                 num_layers, dropout):
        """
        num_classes: 最后输出的文字有多少种可能，也就是整个文本有多少不同的汉字
        embed_dim: 词嵌入维度
        hidden_size: RNN 的节点个数
        num_layers: RNN 的层数
        dropout: dropout的比例
        """
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.word_to_vec = None #todo: 构建词向量
        self.rnn = None #todo: 构建RNN
        self.project = None #todo: 构建最后的分类线性层

    def forward(self, x, hs=None):
        """
        x: 模型的输入，一串任意长度的序列
        hs: 从前面传过来的隐藏状态
        """
        batch = x.shape[0]
        # 如果没有传入隐藏状态，比如第一步，那么构建一个全零的隐藏状态作为初始值
        if hs is None:
            hs = Variable(
                torch.zeros(self.num_layers, batch, self.hidden_size))
            if use_gpu:
                hs = hs.cuda()
        
        word_embed = None  #todo: 将输入传入词嵌入
        word_embed = None  #todo: 改变词嵌入输出的形状，以适应 RNN 的输入
        out, h0 = None  #todo: 将输入和隐藏状态输入 RNN
        
        out = None #todo: 改变 RNN 输出的形状，将 (length, batch_size, hidden_size) --> (batch_size, length, hidden_size)
        out = None #todo: 使用 contiguous 进行连续化
        out = None #todo: 将 (batch_size, length, hidden_size) --> (batch_size x length, hidden_size) 以传入最后的线性层做分类
        out = None #todo: 传入线性层做分类
        return out, h0

### 训练模型
在训练模型的时候，我们知道这是一个分类问题，所以可以使用交叉熵作为 loss 函数，在语言模型中，我们通常使用一个新的指标来评估结果，这个指标叫做困惑度(perplexity)，可以简单地看作对交叉熵取指数，这样其范围就是 $[1, +\infty]$，也是越小越好。

另外，我们前面讲过 RNN 存在着梯度爆炸的问题，所以我们需要进行梯度裁剪，在 pytorch 中使用 `torch.nn.utils.clip_grad_norm` 就能够轻松实现，可以查看[文档](http://pytorch.org/docs/0.3.1/nn.html?highlight=clip_grad#torch.nn.utils.clip_grad_norm)

In [ ]:
from torch.utils.data import DataLoader

#todo: 设置这些参数
batch_size = None
epochs = None
lr = None

embed_dim = None
hidden_size = None
num_layers = None
dropout = None


model = CharRNN(convert.vocab_size, embed_dim, hidden_size, num_layers, dropout)

if use_gpu:
    model = model.cuda()

#todo: 使用交叉熵作为分类的损失函数

#todo: 使用 Adam 作为优化器


train_data = DataLoader(train_set, batch_size, True, num_workers=4)

In [ ]:
def train(model, train_data, epochs, print_freq):
    for e in range(epochs):
        train_loss = 0
        for data in train_data:
            x, y = data
            
            #todo: 将 y 转换成整形，也就是 LongTensor
            
            #todo: 将 y 的形状由 (batch_size, n_step) --> (batch_size x n_step, )
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
            x, y = Variable(x), Variable(y)

            #todo: 前向传播
            
            #todo: 计算 loss
            loss = None
            
            #todo: 反向传播
            
            #todo: 梯度裁剪
            
            #todo: 更新参数
            

            train_loss += loss.data[0]
        if (e + 1) % print_freq == 0:
            print('epoch: {}, perplexity is: {:.3f}'.format(e+1, np.exp(train_loss / len(train_data))))

In [ ]:
train(model, train_data, epochs, 5)

当训练完模型之后，可以达到一个比较小的困惑度，下面我们就可以开始生成文本了。

### 生成文本
生成文本的过程非常简单，前面已将讲过了，给定了开始的字符，然后不断向后生成字符，将生成的字符作为新的输入再传入网络。

这里需要注意的是，为了增加更多的随机性，我们会在预测的概率最高的前五个里面依据他们的概率来进行随机选择，这里的选择函数已经实现如下`pick_top_n`

In [ ]:
def pick_top_n(preds, top_n=5):
    top_pred_prob, top_pred_label = torch.topk(preds, top_n, 1)
    top_pred_prob /= torch.sum(top_pred_prob)
    top_pred_prob = top_pred_prob.squeeze(0).cpu().numpy()
    top_pred_label = top_pred_label.squeeze(0).cpu().numpy()
    c = np.random.choice(top_pred_label, size=1, p=top_pred_prob)
    return c

In [ ]:
# 开始的一段预热字符串
begin = '天青色等烟雨'
# 生成序列的长度
text_len = 30

model = model.eval()
samples = [convert.word_to_int(c) for c in begin] # 将开始的字符串转换成数字
input_txt = torch.LongTensor(samples)[None] # 将其转换成 LongTensor

print('输入的预热字符串形状, batch_size: {}, length: {}'.format(input_txt.shape[0], input_txt.shape[1]))

In [ ]:
if use_gpu:
    input_txt = input_txt.cuda()
    
input_txt = Variable(input_txt)
_, init_state = model(input_txt)

result = samples # 用来记录所有的结果

model_input = None #todo: 选择 input_txt 的最后一个作为后面的输入，输入的形状是 (1, 1)

In [ ]:
for i in range(text_len):
    out, init_state = None  #todo: 进行一次文本生成，生成一个字符
    pred =                  #todo: 随机选择概率最大的5个结果之一
    model_input =           #todo: 将得到结果作为下一步生成的输入，形状是 (1, 1)
    
    if use_gpu:
        model_input = model_input.cuda()
    result.append(pred[0]) # 在结果记录中记录

In [ ]:
text = convert.arr_to_text(result) # 通过 convert 将数字转换成文字
print('生成的文本是: {}'.format(text))